# Experiment 05: Data augmentation

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e05-8', 'e05-6', 'e05-4', 'e05-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = True         # Step 2 of the training process
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e05'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
    
# Data augmentation
AUGMENT_FLIP = True # both up/down and left/right flips
AUGMENT_ROTATE = True # 90 degree rotations
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e05-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e05-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e05-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e05-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e05-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e05-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e05-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e05-8': {None: None},                                   # not enough bands in GE01
                         'e05-6': {None: None},                                    # not enough bands in GE01
                         'e05-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e05-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-pre_20210319-092729/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-gan_20210321-101445/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
if PRE_EVALUATE_HISTORY and GAN_EVALUATE_HISTORY:
    raise ValueError

EVAL_STEPS_PER_EPOCH = 2000
EVAL_N_EPOCHS = 400
EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 1
    EVAL_PREFIX = EXPERIMENT + '-pre-'
elif GAN_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 0
    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: False
NIQE: False
Perceptual Index (PI): False


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129221, 'val': 8113, 'test': 9293}
Total number of tiles: 146627


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 341.3 and sd 128.4 from json file @ data/toulon-laspezia-tiles/e05/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-' 
                                                  + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test],
                                   augment_flip=True,
                                   augment_rotate=True
                                  )
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2006480509994506
Output (float) min, max: -0.2000617970682984 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

16 images from train-WV02 will be logged at each epoch
8 images from val-WV02 will be logged at each epoch
8 images from val-GE01 will be logged at each epoch
Callbacks: [<modules.logging.SimpleEsrganCheckpoint object at 0x000002BC60D126C8>, <tensorflow.python.keras.callbacks.TensorBoard object at 0x000002BC84177CC8>, <modules.logging.MultipleValSetsCallback object at 0x000002BCAAF89108>, <modules.logging.LrHrSrImageCallback object at 0x000002BC841B5288>, <modules.logging.LrHrSrImageCallback object at 0x000002BCAB1DE648>]
Epoch 1/400
1000/1000 [==============================] - 463s 463ms/step - G_pixel_loss: 1.2970e-04 - G_perceptual_loss: 2.1797 - G_generator_loss: 0.0232 - G_loss_total: 2.2031 - D_loss_total: 0.1640 - PSNR: 37.5345 - SSIM: 0.8803
Epoch 2/400
1000/1000 [==============================] - 465s 465ms/step - G_pixel_loss: 1.2783e-04 - G_perceptual_loss: 2.1302 - G_generator_loss: 0.0438 - G_loss_total: 2.1742 - D_loss_total: 0.0121 - PSNR: 37.1666 - SSIM: 0.8856
Epoch 3/

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2994e-04 - G_perceptual_loss: 2.0027 - G_generator_loss: 0.0467 - G_loss_total: 2.0495 - D_loss_total: 0.0459 - PSNR: 37.2694 - SSIM: 0.8761
Epoch 34/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2955e-04 - G_perceptual_loss: 1.9919 - G_generator_loss: 0.0435 - G_loss_total: 2.0356 - D_loss_total: 0.0516 - PSNR: 37.3361 - SSIM: 0.8759
Epoch 35/400
1000/1000 [==============================] - 459s 459ms/step - G_pixel_loss: 1.2858e-04 - G_perceptual_loss: 1.9829 - G_generator_loss: 0.0453 - G_loss_total: 2.0284 - D_loss_total: 0.0536 - PSNR: 37.3079 - SSIM: 0.8773
Epoch 36/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3002e-04 - G_perceptual_loss: 1.9996 - G_generator_loss: 0.0501 - G_loss_total: 2.0498 - D_loss_total: 0.0446 - PSNR: 37.3200 - SSIM: 0.8756
Epoch 37/400
1000/1000 [==============================] - 460s 460ms/step - G_pix

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2880e-04 - G_perceptual_loss: 1.9379 - G_generator_loss: 0.0395 - G_loss_total: 1.9775 - D_loss_total: 0.0638 - PSNR: 37.3611 - SSIM: 0.8762
Epoch 69/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2871e-04 - G_perceptual_loss: 1.9507 - G_generator_loss: 0.0460 - G_loss_total: 1.9968 - D_loss_total: 0.0304 - PSNR: 37.3142 - SSIM: 0.8765
Epoch 70/400
1000/1000 [==============================] - 459s 459ms/step - G_pixel_loss: 1.2919e-04 - G_perceptual_loss: 1.9530 - G_generator_loss: 0.0481 - G_loss_total: 2.0012 - D_loss_total: 0.0390 - PSNR: 37.3232 - SSIM: 0.8760
Epoch 71/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3033e-04 - G_perceptual_loss: 1.9581 - G_generator_loss: 0.0459 - G_loss_total: 2.0042 - D_loss_total: 0.0528 - PSNR: 37.1671 - SSIM: 0.8736
Epoch 72/400
1000/1000 [==============================] - 460s 460ms/step - G_pix

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3115e-04 - G_perceptual_loss: 1.9516 - G_generator_loss: 0.0499 - G_loss_total: 2.0017 - D_loss_total: 0.0401 - PSNR: 37.1392 - SSIM: 0.8730
Epoch 104/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3045e-04 - G_perceptual_loss: 1.9352 - G_generator_loss: 0.0531 - G_loss_total: 1.9884 - D_loss_total: 0.0444 - PSNR: 37.2169 - SSIM: 0.8740
Epoch 105/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3087e-04 - G_perceptual_loss: 1.9393 - G_generator_loss: 0.0502 - G_loss_total: 1.9896 - D_loss_total: 0.0455 - PSNR: 37.2100 - SSIM: 0.8744
Epoch 106/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2905e-04 - G_perceptual_loss: 1.9282 - G_generator_loss: 0.0494 - G_loss_total: 1.9777 - D_loss_total: 0.0343 - PSNR: 37.3007 - SSIM: 0.8762
Epoch 107/400
1000/1000 [==============================] - 460s 460ms/step - G

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2855e-04 - G_perceptual_loss: 1.9024 - G_generator_loss: 0.0482 - G_loss_total: 1.9507 - D_loss_total: 0.0382 - PSNR: 37.3649 - SSIM: 0.8761
Epoch 139/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2929e-04 - G_perceptual_loss: 1.9092 - G_generator_loss: 0.0472 - G_loss_total: 1.9565 - D_loss_total: 0.0424 - PSNR: 37.2858 - SSIM: 0.8753
Epoch 140/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.3006e-04 - G_perceptual_loss: 1.9098 - G_generator_loss: 0.0513 - G_loss_total: 1.9613 - D_loss_total: 0.0338 - PSNR: 37.1941 - SSIM: 0.8740
Epoch 141/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2930e-04 - G_perceptual_loss: 1.9126 - G_generator_loss: 0.0508 - G_loss_total: 1.9636 - D_loss_total: 0.0436 - PSNR: 37.2475 - SSIM: 0.8757
Epoch 142/400
1000/1000 [==============================] - 460s 460ms/step - G

...

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2723e-04 - G_perceptual_loss: 1.8520 - G_generator_loss: 0.0512 - G_loss_total: 1.9033 - D_loss_total: 0.0246 - PSNR: 37.4119 - SSIM: 0.8776
Epoch 244/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2889e-04 - G_perceptual_loss: 1.8461 - G_generator_loss: 0.0482 - G_loss_total: 1.8944 - D_loss_total: 0.0496 - PSNR: 37.2792 - SSIM: 0.8759
Epoch 245/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2808e-04 - G_perceptual_loss: 1.8494 - G_generator_loss: 0.0514 - G_loss_total: 1.9009 - D_loss_total: 0.0346 - PSNR: 37.3805 - SSIM: 0.8761
Epoch 246/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2955e-04 - G_perceptual_loss: 1.8734 - G_generator_loss: 0.0570 - G_loss_total: 1.9306 - D_loss_total: 0.0227 - PSNR: 37.2478 - SSIM: 0.8752
Epoch 247/400
1000/1000 [==============================] - 460s 460ms/step - G

1000/1000 [==============================] - 463s 463ms/step - G_pixel_loss: 1.2856e-04 - G_perceptual_loss: 1.8519 - G_generator_loss: 0.0500 - G_loss_total: 1.9020 - D_loss_total: 0.0393 - PSNR: 37.3093 - SSIM: 0.8771
Epoch 279/400
1000/1000 [==============================] - 463s 463ms/step - G_pixel_loss: 1.2850e-04 - G_perceptual_loss: 1.8476 - G_generator_loss: 0.0554 - G_loss_total: 1.9032 - D_loss_total: 0.0291 - PSNR: 37.2785 - SSIM: 0.8770
Epoch 280/400
1000/1000 [==============================] - 463s 463ms/step - G_pixel_loss: 1.2918e-04 - G_perceptual_loss: 1.8546 - G_generator_loss: 0.0572 - G_loss_total: 1.9120 - D_loss_total: 0.0431 - PSNR: 37.3251 - SSIM: 0.8761
Epoch 281/400
1000/1000 [==============================] - 464s 464ms/step - G_pixel_loss: 1.2782e-04 - G_perceptual_loss: 1.8398 - G_generator_loss: 0.0525 - G_loss_total: 1.8924 - D_loss_total: 0.0419 - PSNR: 37.3789 - SSIM: 0.8764
Epoch 282/400
1000/1000 [==============================] - 463s 463ms/step - G

1000/1000 [==============================] - 459s 459ms/step - G_pixel_loss: 1.2887e-04 - G_perceptual_loss: 1.8432 - G_generator_loss: 0.0590 - G_loss_total: 1.9023 - D_loss_total: 0.0246 - PSNR: 37.3147 - SSIM: 0.8758
Epoch 314/400
1000/1000 [==============================] - 459s 459ms/step - G_pixel_loss: 1.2905e-04 - G_perceptual_loss: 1.8494 - G_generator_loss: 0.0526 - G_loss_total: 1.9021 - D_loss_total: 0.0559 - PSNR: 37.2487 - SSIM: 0.8756
Epoch 315/400
1000/1000 [==============================] - 459s 459ms/step - G_pixel_loss: 1.2805e-04 - G_perceptual_loss: 1.8446 - G_generator_loss: 0.0565 - G_loss_total: 1.9012 - D_loss_total: 0.0186 - PSNR: 37.3633 - SSIM: 0.8765
Epoch 316/400
1000/1000 [==============================] - 458s 458ms/step - G_pixel_loss: 1.2915e-04 - G_perceptual_loss: 1.8435 - G_generator_loss: 0.0548 - G_loss_total: 1.8984 - D_loss_total: 0.0369 - PSNR: 37.2901 - SSIM: 0.8748
Epoch 317/400
1000/1000 [==============================] - 458s 458ms/step - G

1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2702e-04 - G_perceptual_loss: 1.8172 - G_generator_loss: 0.0562 - G_loss_total: 1.8735 - D_loss_total: 0.0303 - PSNR: 37.4228 - SSIM: 0.8775
Epoch 349/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2858e-04 - G_perceptual_loss: 1.8369 - G_generator_loss: 0.0516 - G_loss_total: 1.8887 - D_loss_total: 0.0344 - PSNR: 37.2427 - SSIM: 0.8759
Epoch 350/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2748e-04 - G_perceptual_loss: 1.8168 - G_generator_loss: 0.0536 - G_loss_total: 1.8705 - D_loss_total: 0.0275 - PSNR: 37.4385 - SSIM: 0.8771
Epoch 351/400
1000/1000 [==============================] - 460s 460ms/step - G_pixel_loss: 1.2861e-04 - G_perceptual_loss: 1.8401 - G_generator_loss: 0.0606 - G_loss_total: 1.9008 - D_loss_total: 0.0239 - PSNR: 37.2693 - SSIM: 0.8762
Epoch 352/400
1000/1000 [==============================] - 460s 460ms/step - G

1000/1000 [==============================] - 462s 462ms/step - G_pixel_loss: 1.2776e-04 - G_perceptual_loss: 1.8220 - G_generator_loss: 0.0567 - G_loss_total: 1.8787 - D_loss_total: 0.0365 - PSNR: 37.3613 - SSIM: 0.8772
Epoch 384/400
1000/1000 [==============================] - 462s 462ms/step - G_pixel_loss: 1.2799e-04 - G_perceptual_loss: 1.8141 - G_generator_loss: 0.0563 - G_loss_total: 1.8706 - D_loss_total: 0.0246 - PSNR: 37.3799 - SSIM: 0.8756
Epoch 385/400
1000/1000 [==============================] - 461s 461ms/step - G_pixel_loss: 1.2805e-04 - G_perceptual_loss: 1.8223 - G_generator_loss: 0.0553 - G_loss_total: 1.8777 - D_loss_total: 0.0319 - PSNR: 37.3200 - SSIM: 0.8767
Epoch 386/400
1000/1000 [==============================] - 462s 462ms/step - G_pixel_loss: 1.2715e-04 - G_perceptual_loss: 1.8076 - G_generator_loss: 0.0557 - G_loss_total: 1.8634 - D_loss_total: 0.0329 - PSNR: 37.4125 - SSIM: 0.8775
Epoch 387/400
1000/1000 [==============================] - 461s 461ms/step - G

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
val_or_test = 'val'

# Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
if METRIC_MA:
    PRE_GAN = ['gan']
else:
    PRE_GAN = ['pre', 'gan']
    
for pre_gan in PRE_GAN:
    for sensor in SENSORS[val_or_test]:
        if sensor == 'GE01':
            band_indices = MS_BANDS_GE01_IDXS
        elif sensor == 'WV02':
            band_indices = MS_BANDS_WV02_IDXS
        if pre_gan == 'pre':
            gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
        else:
            gan_model.G.load_weights(GAN_WEIGHTS_PATH)

        start = time.time()
        results_df = esrgan_evaluate(model=gan_model, 
                                     dataset=ds_val[sensor], 
                                     steps='all', 
                                     per_image=True, 
                                     write_csv=True,
                                     csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                  + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                     verbose=1
                                    )
        end = time.time()
        print(str((end - start) / 60), 'minutes')

Computed 8 images in  6.1780102252960205 seconds
Last image: {'G_pixel_loss': 8.639927546028048e-05, 'G_perceptual_loss': 1.7869559526443481, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 36.124629974365234, 'SSIM': 0.9352430701255798}
Computed 8 images in  1.3527257442474365 seconds
Last image: {'G_pixel_loss': 2.7937423510593362e-05, 'G_perceptual_loss': 1.016118049621582, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 40.00178146362305, 'SSIM': 0.9820539951324463}
Computed 8 images in  1.3308444023132324 seconds
Last image: {'G_pixel_loss': 0.00017985999875236303, 'G_perceptual_loss': 4.469176292419434, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.822784423828125, 'SSIM': 0.8687350749969482}
Computed 8 images in  1.346942663192749 seconds
Last image: {'G_pixel_loss': 0.00010348198702558875, 'G_perceptual_loss': 3.131742238998413, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0

Computed 8 images in  1.3614740371704102 seconds
Last image: {'G_pixel_loss': 0.00012892315862700343, 'G_perceptual_loss': 3.9248104095458984, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.110145568847656, 'SSIM': 0.866102397441864}
Computed 8 images in  1.3622899055480957 seconds
Last image: {'G_pixel_loss': 0.00014829542487859726, 'G_perceptual_loss': 4.008449554443359, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 32.780033111572266, 'SSIM': 0.8845722079277039}
Computed 8 images in  1.393869161605835 seconds
Last image: {'G_pixel_loss': 0.00010454483708599582, 'G_perceptual_loss': 3.0086252689361572, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.049835205078125, 'SSIM': 0.9113633632659912}
Computed 8 images in  1.393514633178711 seconds
Last image: {'G_pixel_loss': 3.7335332308430225e-05, 'G_perceptual_loss': 0.8579031825065613, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0

Computed 8 images in  1.3321425914764404 seconds
Last image: {'G_pixel_loss': 4.870443444815464e-05, 'G_perceptual_loss': 1.480384349822998, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 40.163204193115234, 'SSIM': 0.9624616503715515}
Computed 8 images in  1.362847089767456 seconds
Last image: {'G_pixel_loss': 0.00018557909061200917, 'G_perceptual_loss': 4.0389084815979, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.272079467773438, 'SSIM': 0.8142363429069519}
Computed 8 images in  1.4083020687103271 seconds
Last image: {'G_pixel_loss': 9.408740152139217e-05, 'G_perceptual_loss': 1.5539990663528442, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 37.12030792236328, 'SSIM': 0.9063317179679871}
Computed 8 images in  1.36185622215271 seconds
Last image: {'G_pixel_loss': 9.796919766813517e-05, 'G_perceptual_loss': 1.388088345527649, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSN

Computed 8 images in  1.3310439586639404 seconds
Last image: {'G_pixel_loss': 5.179033905733377e-05, 'G_perceptual_loss': 1.2885843515396118, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 41.857177734375, 'SSIM': 0.957808792591095}
Computed 8 images in  1.361799716949463 seconds
Last image: {'G_pixel_loss': 0.00010942272638203576, 'G_perceptual_loss': 1.9122370481491089, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 36.26304626464844, 'SSIM': 0.8553647994995117}
Computed 8 images in  1.3308830261230469 seconds
Last image: {'G_pixel_loss': 0.00012353481724858284, 'G_perceptual_loss': 1.9961942434310913, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.78120422363281, 'SSIM': 0.8198796510696411}
Computed 8 images in  1.3474552631378174 seconds
Last image: {'G_pixel_loss': 6.744323763996363e-05, 'G_perceptual_loss': 1.8980122804641724, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, '

Computed 8 images in  1.3965966701507568 seconds
Last image: {'G_pixel_loss': 0.0004975013434886932, 'G_perceptual_loss': 1.0765794515609741, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.531484603881836, 'SSIM': 0.7883098125457764}
Computed 8 images in  1.3294694423675537 seconds
Last image: {'G_pixel_loss': 0.0005097007378935814, 'G_perceptual_loss': 1.1011042594909668, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.327287673950195, 'SSIM': 0.7617558836936951}
Computed 8 images in  1.3306150436401367 seconds
Last image: {'G_pixel_loss': 0.0003588981053326279, 'G_perceptual_loss': 1.1315863132476807, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.791683197021484, 'SSIM': 0.6377419829368591}
Computed 8 images in  1.3297784328460693 seconds
Last image: {'G_pixel_loss': 0.0001452204742236063, 'G_perceptual_loss': 3.7754104137420654, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0

...

Computed 8 images in  1.4250571727752686 seconds
Last image: {'G_pixel_loss': 0.00017161072173621505, 'G_perceptual_loss': 3.6862502098083496, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.7167911529541, 'SSIM': 0.847510576248169}
Computed 8 images in  1.3472926616668701 seconds
Last image: {'G_pixel_loss': 0.00022549684217665344, 'G_perceptual_loss': 4.747840404510498, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.999649047851562, 'SSIM': 0.809638261795044}
Computed 8 images in  1.346496820449829 seconds
Last image: {'G_pixel_loss': 0.0002949587069451809, 'G_perceptual_loss': 5.457148551940918, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.28080940246582, 'SSIM': 0.6939656734466553}
Computed 8 images in  1.3147776126861572 seconds
Last image: {'G_pixel_loss': 3.076018765568733e-05, 'G_perceptual_loss': 0.7802600860595703, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'P

Computed 8 images in  1.330779790878296 seconds
Last image: {'G_pixel_loss': 0.00038482772652059793, 'G_perceptual_loss': 4.270571708679199, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.25334358215332, 'SSIM': 0.6474575996398926}
Computed 8 images in  1.3615162372589111 seconds
Last image: {'G_pixel_loss': 0.0004858844622503966, 'G_perceptual_loss': 2.7566559314727783, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.540428161621094, 'SSIM': 0.4522065222263336}
Computed 8 images in  1.407423496246338 seconds
Last image: {'G_pixel_loss': 0.0003596721508074552, 'G_perceptual_loss': 2.922212839126587, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.154380798339844, 'SSIM': 0.7963126301765442}
Computed 8 images in  1.3147306442260742 seconds
Last image: {'G_pixel_loss': 0.0004983549006283283, 'G_perceptual_loss': 2.2754831314086914, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 

Computed 8 images in  1.4087696075439453 seconds
Last image: {'G_pixel_loss': 0.0004210102488286793, 'G_perceptual_loss': 3.35577130317688, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.870075225830078, 'SSIM': 0.7169064879417419}
Computed 8 images in  1.3631565570831299 seconds
Last image: {'G_pixel_loss': 0.00042462622513994575, 'G_perceptual_loss': 4.203309535980225, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.428573608398438, 'SSIM': 0.5822979211807251}
Computed 8 images in  1.3628063201904297 seconds
Last image: {'G_pixel_loss': 0.0002848833682946861, 'G_perceptual_loss': 3.00441312789917, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.132478713989258, 'SSIM': 0.6755591034889221}
Computed 8 images in  1.3537230491638184 seconds
Last image: {'G_pixel_loss': 0.0004710577195510268, 'G_perceptual_loss': 4.334820747375488, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, '

Computed 8 images in  1.3441009521484375 seconds
Last image: {'G_pixel_loss': 0.000912923424039036, 'G_perceptual_loss': 7.407924652099609, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 20.28165054321289, 'SSIM': 0.5766298770904541}
Computed 8 images in  1.3466780185699463 seconds
Last image: {'G_pixel_loss': 0.0006054093828424811, 'G_perceptual_loss': 6.264618396759033, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 23.426347732543945, 'SSIM': 0.7495761513710022}
Computed 8 images in  1.3156461715698242 seconds
Last image: {'G_pixel_loss': 0.0010868326062336564, 'G_perceptual_loss': 9.5186185836792, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 18.587778091430664, 'SSIM': 0.45606204867362976}
Computed 8 images in  1.3147633075714111 seconds
Last image: {'G_pixel_loss': 0.0003316798829473555, 'G_perceptual_loss': 0.4424448609352112, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'P

NameError: name 'GAN_WEIGHTS_PATH' is not defined

### 8.3 Evaluate every kth epoch

### 8.4 Comparison plots

In [ ]:
idx = 10
import plotly.express as px
batch = next(iter(ds_train['WV02']))
ms = batch[0][1][idx,:,:,:3]
pan = batch[1][1][idx,:,:,:]
px.imshow(stretch_img(ms))
px.imshow(stretch_img(pan)[:,:,0], color_continuous_scale='gray')